<a href="https://colab.research.google.com/github/AtaruOhto/pytorch_learning/blob/master/001_pytorch_basis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
PyTorch Tensorの各種メソッド
"""

import torch
import numpy as np

In [ ]:
empties = torch.empty(5,3)
print("empties", empties)

In [22]:
print(empties.size())

torch.Size([5, 3])


In [ ]:
randoms = torch.rand(4, 3)
print("randoms", randoms)

In [ ]:
longs = torch.zeros(5, 2, dtype=torch.long)
print("longs",longs)

In [ ]:
ts = torch.tensor([1.2, 3.3])
print("ts",ts)

In [ ]:
new_ones = torch.tensor([1, 2]).new_ones(5, 3, dtype=torch.double)
print("new_ones", new_ones)

In [ ]:
randn_likes = torch.randn_like(torch.tensor([[1,2,3], [4,5,6]], dtype=torch.float))
print("randn_like", randn_likes)

In [28]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)
z = torch.empty(5, 3)
print("x + y = ", torch.add(x, y, out=z))
print(z)
y.add_(x)




x + y =  tensor([[1.1705, 0.8212, 1.3312],
        [0.2151, 0.6536, 1.3623],
        [0.2499, 1.4627, 1.1514],
        [1.3153, 1.5213, 1.1296],
        [1.4744, 1.4722, 0.3519]])
tensor([[1.1705, 0.8212, 1.3312],
        [0.2151, 0.6536, 1.3623],
        [0.2499, 1.4627, 1.1514],
        [1.3153, 1.5213, 1.1296],
        [1.4744, 1.4722, 0.3519]])


tensor([[1.1705, 0.8212, 1.3312],
        [0.2151, 0.6536, 1.3623],
        [0.2499, 1.4627, 1.1514],
        [1.3153, 1.5213, 1.1296],
        [1.4744, 1.4722, 0.3519]])

In [4]:
# Numpy to PyTorch Tensor
a = np.ones(5)
b = torch.from_numpy(a)
np.add(1, 1, out=a)
print(a)

[2. 2. 2. 2. 2.]


In [26]:
# 自動微分 (autograd)

""" 
1: PyTorchではrequire_grad=Trueにすることで、傾きをトラッキングすることができる。
2: backward()関数で傾き (微分) を取得することができ、 grad プロパティにその値を保持する。
3: 自動微分を停止するには detach()関数を呼び出す or requires_grad_ or with torch.no_grad() でも停止可能
"""

x = torch.ones(2, 2, requires_grad=True)
y = x + 2 * 2

print("yの値", y)
print("記憶している計算式", y.grad_fn)
print("自動微分が有効か?",y.requires_grad)


yの値 tensor([[5., 5.],
        [5., 5.]], grad_fn=<AddBackward0>)
記憶している計算式 <AddBackward0 object at 0x7f5b200ad278>
自動微分が有効か? True


In [38]:
x1 = torch.tensor([[5, 5], [10, 10]], dtype=float, requires_grad=True)
x2 = x1 + 3
x3 = x2 * 3
x4 = x3.mean()

# 微分する
x4.backward()
print("x1の微分値", x１.grad)


x1の微分値 tensor([[0.7500, 0.7500],
        [0.7500, 0.7500]], dtype=torch.float64)


In [ ]:
import torch

class SampleData:
  """
    データ: X, y
    オリジナルの重み: original_weight
    を準備する。
  """
  def __init__(self):
    # 元々の重み
    original_weight = torch.Tensor([1, 2, 3])
    # データ形式: torch.Size([3])

    # Xのデータ準備 
    # データ形式: torch.Size([100, 3])
    X = torch.cat([torch.ones(100, 1), torch.randn(100, 2)], 1)

    # データと重みの内積を計算する。
    dot = torch.mv(X, original_weight)
    
    # 内積に乱数を足してyとする。
    # データ形式: torch.Size([100])
    y = dot + torch.randn(100) * 0.5    

    self.X = X
    self.y = y
    self.original_weight = original_weight

def exec_gradient_descent(weight, X, y):
  # 学習率を0.1とする
  eta = 0.1

  # 損失関数のログ (下のループ中で誤差を格納していく)
  losses = []

  # 50回ループさせて、勾配降下法でパラメータを最適化する
  for epoc in range(50):
    # 前のループ中のbackward()メソッドで計算された勾配の値を削除
    weight.grad = None
    
    # yの予測値を計算する
    y_pred = torch.mv(X, weight)

    # 平均二乗誤差Mean Square Error を計算 (実際の値と予測値のズレがどれだけあるかの指標のひとつ)
    # 誤差が小さいほどモデルの性能が高い。
    loss = torch.mean((y - y_pred) ** 2)

    # 誤差逆伝播して、勾配を計算。
    # weightのrequires_grad=Trueにしているので、自動で微分が計算される。
    loss.backward()

    # 重みを更新する。
    weight.data = weight.data - eta * weight.grad.data

    # ループの終わりにlossを格納する。誤差が小さくなっていくのを確認。
    losses.append(loss.item())
  return weight

data = SampleData()

# Tensorを乱数で初期化して、重みとする。
# (以下でこの重みをループ中で、「実際のyと予測値y_predとの誤差を小さくするように」更新していく。)
# requires_gradをTrueにすることで重みの自動微分を有効にする。
# https://pytorch.org/docs/stable/notes/autograd.html#requires-grad
weight_will_updated = torch.randn(3, requires_grad=True)

# 乱数で作った重み (weight_will_updated) を更新する。
# (Xの計算式から、yの値を正しく予測するように重みを更新する。)
weight = exec_gradient_descent(weight_will_updated, data.X, data.y)

# matplotlibで誤差 (loss) の推移を描画する。
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(losses)

# 元々の重みと、更新された重みの差分を見る。
# 差分は極小になっている。
print(torch.abs(data.original_weight - weight))